# **Networth Tracker**

## **Report**

We generate a report for each available year.

### **Libraries**

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go

print('Libraries Imported')

### **Data**

First, we import the data.

In [ ]:
DATA_PATH = Path('data') / 'raw' / 'daniele' / 'networth' / 'data.csv'

In [ ]:
# Read the CSV file into a list of lines
with Path.open(DATA_PATH, encoding='utf-8') as f:
    lines = f.readlines()

# Find the line number where 'Mese' appears to identify the header rows
for i, line in enumerate(lines):
    if 'Mese' in line:
        header_line_number = i
        break

# Read the data starting from the header line, using the next two lines as headers
dataframe = pd.read_csv(
    DATA_PATH,
    skiprows=header_line_number + 1,
    encoding='utf-8',
)

# Remove columns that are not needed
dataframe = dataframe.iloc[:, 2:-9].dropna()

# Rename the columns
cols_name = [
    'Mese',
    'Liquidità',
    'Fondo Pensione (Lordo)',
    'Fondo Pensione (Netto)',
    'Obbligazioni (Lordo)',
    'Obbligazioni (Netto)',
    'BIT:SWDA (ETF)',
    'BIT:EIMI (ETF)',
    'BIT:SGLD (ETC)',
    'Azioni ed ETF (Lordo)',
    'Azioni ed ETF (Netto)',
]
dataframe.columns = cols_name

# Define a mapping from Italian month names to month numbers
month_map = {
    'gennaio': 1,
    'febbraio': 2,
    'marzo': 3,
    'aprile': 4,
    'maggio': 5,
    'giugno': 6,
    'luglio': 7,
    'agosto': 8,
    'settembre': 9,
    'ottobre': 10,
    'novembre': 11,
    'dicembre': 12,
}


# Function to parse Italian date strings
def parse_italian_date(date_str):
    """Parse an Italian date string in the format 'day month year' into a pandas Timestamp."""
    parts_to_find = 3
    try:
        date_str = str(date_str).strip().lower()
        parts = date_str.split()
        if len(parts) != parts_to_find:
            return pd.NaT
        day = int(parts[0])
        month_str = parts[1]
        year = int(parts[2])
        month = month_map.get(month_str)
        if month is None:
            return pd.NaT
        return pd.Timestamp(year=year, month=month, day=day)
    except:  # noqa
        return pd.NaT


# Identify the 'Mese' column
# Assuming 'Mese' is the first column, adjust if necessary
mese_columns = [col for col in dataframe.columns if 'Mese' in col]
if not mese_columns:
    exc_desc = "No column containing 'Mese' found."
    raise ValueError(exc_desc)
mese_column = mese_columns[0]

# Apply the date parsing function
dataframe[mese_column] = dataframe[mese_column].apply(parse_italian_date)


# Define functions to clean currency and percentage values
def parse_currency(value):
    """Parse a currency value in Italian format."""
    try:
        if pd.isna(value):
            return np.nan
        value = str(value)
        # Remove euro symbol, spaces, and handle thousand separators
        value = value.replace('€', '').replace(' ', '').replace('.', '').replace(',', '.')
        return float(value)
    except:  # noqa
        return np.nan


# Apply the parsing functions to the relevant columns
for col in cols_name[1:]:
    dataframe[col] = dataframe[col].apply(parse_currency)

# Show results
dataframe

### **Dataviz**

In [ ]:
# Convert 'Mese' to datetime for better handling, keep only month and year
dataframe['Mese'] = pd.to_datetime(dataframe['Mese'], format='%Y-%m')
dataframe['Anno_Mese_str'] = dataframe['Mese'].dt.strftime('%Y-%m')

# Calculate Tax for Fondo Pensione, Obbligazioni, and Azioni ed ETF
dataframe['Fondo Pensione Tax'] = (
    dataframe['Fondo Pensione (Lordo)'] - dataframe['Fondo Pensione (Netto)']
)
dataframe['Obbligazioni Tax'] = (
    dataframe['Obbligazioni (Lordo)'] - dataframe['Obbligazioni (Netto)']
)
dataframe['Azioni ed ETF Tax'] = (
    dataframe['Azioni ed ETF (Lordo)'] - dataframe['Azioni ed ETF (Netto)']
)


# Function to add bar traces with conditional text
MIN_VALUE_FOR_TEXT = 3000


def add_bar_trace(fig, name, x, y, color):
    """Add a bar trace to the figure with conditional text."""
    text = [f'{val:.0f} €' if val > MIN_VALUE_FOR_TEXT else '' for val in y]
    fig.add_trace(
        go.Bar(
            name=name,
            x=x,
            y=y,
            marker_color=color,
            text=text,
            textposition='inside',
            insidetextanchor='middle',  # Center the text horizontally
            textfont={'size': 9},
        )
    )


# Initialize the figure
fig = go.Figure()

# Add Liquidità
add_bar_trace(
    fig,
    name='Liquidità',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Liquidità'],
    color='rgba(50, 171, 96, 0.7)',
)

# Add Fondo Pensione Netto and Tax
add_bar_trace(
    fig,
    name='Fondo Pensione Netto',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Fondo Pensione (Netto)'],
    color='rgba(26, 118, 255, 0.7)',
)

add_bar_trace(
    fig,
    name='Fondo Pensione Tax',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Fondo Pensione Tax'],
    color='rgba(26, 118, 255, 0.3)',
)

# Add Obbligazioni Netto and Tax
add_bar_trace(
    fig,
    name='Obbligazioni Netto',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Obbligazioni (Netto)'],
    color='rgba(247, 238, 127, 0.7)',
)

add_bar_trace(
    fig,
    name='Obbligazioni Tax',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Obbligazioni Tax'],
    color='rgba(247, 238, 127, 0.3)',
)

# Add Azioni ed ETF Netto and Tax
add_bar_trace(
    fig,
    name='Azioni ed ETF Netto',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Azioni ed ETF (Netto)'],
    color='rgba(219, 64, 82, 0.7)',
)

add_bar_trace(
    fig,
    name='Azioni ed ETF Tax',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Azioni ed ETF Tax'],
    color='rgba(219, 64, 82, 0.3)',
)

# Calculate the gross total per month (including taxes)
dataframe['Gross_Total'] = (
    dataframe['Liquidità']
    + dataframe['Fondo Pensione (Netto)']
    + dataframe['Fondo Pensione Tax']
    + dataframe['Obbligazioni (Netto)']
    + dataframe['Obbligazioni Tax']
    + dataframe['Azioni ed ETF (Netto)']
    + dataframe['Azioni ed ETF Tax']
)

# Calculate the net total per month (excluding taxes)
dataframe['Net_Total'] = (
    dataframe['Liquidità']
    + dataframe['Fondo Pensione (Netto)']
    + dataframe['Obbligazioni (Netto)']
    + dataframe['Azioni ed ETF (Netto)']
)

# Define offsets for the labels
offset = dataframe['Gross_Total'].max() * 0.02  # 2% of the max total
net_offset = offset  # Offset for net total labels
gross_offset = offset * 2  # Offset for gross total labels (placed above net total labels)

# Positions for labels
dataframe['Net_Total_Text_Y'] = dataframe['Gross_Total'] + net_offset
dataframe['Gross_Total_Text_Y'] = dataframe['Gross_Total'] + gross_offset

# Add the net total as a text label just above the bar
fig.add_trace(
    go.Scatter(
        x=dataframe['Anno_Mese_str'],
        y=dataframe['Net_Total_Text_Y'],
        mode='text',
        text=dataframe['Net_Total'].apply(lambda x: f'({x:,.0f} €)'),
        textposition='top center',
        textfont={'size': 12, 'color': 'black', 'family': 'Arial'},
        showlegend=False,
    )
)

# Add the gross total as a text label above the net total label
fig.add_trace(
    go.Scatter(
        x=dataframe['Anno_Mese_str'],
        y=dataframe['Gross_Total_Text_Y'],
        mode='text',
        text=dataframe['Gross_Total'].apply(lambda x: f'{x:,.0f} €'),
        textposition='top center',
        textfont={'size': 12, 'color': 'black', 'family': 'Arial'},
        showlegend=False,
    )
)

# Update layout for stacked bars and connected appearance
fig.update_layout(
    barmode='stack',
    title='Portfolio Composition Over Time',
    xaxis_title='Month',
    yaxis_title='Amount (€)',
    legend_title='Categories',
    template='plotly_white',
    xaxis={
        'tickformat': '%Y-%m',
        'tickangle': -45,
        'type': 'category',
    },
    uniformtext={'minsize': 8, 'mode': 'hide'},
    margin={'b': 150, 't': 150},
    width=1980,
    height=1080,
)

# Adjust the y-axis to accommodate the labels
y_max = dataframe['Gross_Total_Text_Y'].max() * 1.05  # Add 5% extra space on top
fig.update_yaxes(range=[0, y_max])

# Adjust the layout to ensure bars are connected
fig.update_xaxes(categoryorder='category ascending')

# Show the figure
fig.show()

In [ ]:
# Convert 'Mese' to datetime for better handling, keep only month and year
dataframe['Mese'] = pd.to_datetime(dataframe['Mese'], format='%Y-%m')
dataframe['Anno_Mese_str'] = dataframe['Mese'].dt.strftime('%Y-%m')

# Calculate Tax for Fondo Pensione, Obbligazioni, and Azioni ed ETF
dataframe['Fondo Pensione Tax'] = (
    dataframe['Fondo Pensione (Lordo)'] - dataframe['Fondo Pensione (Netto)']
)
dataframe['Obbligazioni Tax'] = (
    dataframe['Obbligazioni (Lordo)'] - dataframe['Obbligazioni (Netto)']
)
dataframe['Azioni ed ETF Tax'] = (
    dataframe['Azioni ed ETF (Lordo)'] - dataframe['Azioni ed ETF (Netto)']
)

# Calculate the gross total per month (including taxes)
dataframe['Gross_Total'] = (
    dataframe['Liquidità']
    + dataframe['Fondo Pensione (Netto)']
    + dataframe['Fondo Pensione Tax']
    + dataframe['Obbligazioni (Netto)']
    + dataframe['Obbligazioni Tax']
    + dataframe['Azioni ed ETF (Netto)']
    + dataframe['Azioni ed ETF Tax']
)

# Compute percentage for each component
components = {
    'Liquidità': 'Liquidità',
    'Fondo Pensione Netto': 'Fondo Pensione (Netto)',
    'Fondo Pensione Tax': 'Fondo Pensione Tax',
    'Obbligazioni Netto': 'Obbligazioni (Netto)',
    'Obbligazioni Tax': 'Obbligazioni Tax',
    'Azioni ed ETF Netto': 'Azioni ed ETF (Netto)',
    'Azioni ed ETF Tax': 'Azioni ed ETF Tax',
}

for column in components.values():
    percentage_column = f'{column}_Percentage'
    dataframe[percentage_column] = (dataframe[column] / dataframe['Gross_Total']) * 100

# Function to add bar traces with conditional text
MIN_PERCENT_FOR_TEXT = 3  # Display text for components >3%


def add_bar_trace_percentage(fig, name, x, y, color):
    """Add a bar trace to the figure with conditional text for percentage."""
    text = [f'{val:.1f}%' if val > MIN_PERCENT_FOR_TEXT else '' for val in y]
    fig.add_trace(
        go.Bar(
            name=name,
            x=x,
            y=y,
            marker_color=color,
            text=text,
            textposition='inside',
            insidetextanchor='middle',  # Center the text horizontally
            textfont={'size': 10},
        )
    )


# Initialize the figure
fig = go.Figure()

# Add each component as a percentage trace
add_bar_trace_percentage(
    fig,
    name='Liquidità',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Liquidità_Percentage'],
    color='rgba(50, 171, 96, 0.7)',
)

add_bar_trace_percentage(
    fig,
    name='Fondo Pensione Netto',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Fondo Pensione (Netto)_Percentage'],
    color='rgba(26, 118, 255, 0.7)',
)

add_bar_trace_percentage(
    fig,
    name='Fondo Pensione Tax',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Fondo Pensione Tax_Percentage'],
    color='rgba(26, 118, 255, 0.3)',
)

add_bar_trace_percentage(
    fig,
    name='Obbligazioni Netto',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Obbligazioni (Netto)_Percentage'],
    color='rgba(247, 238, 127, 0.7)',
)

add_bar_trace_percentage(
    fig,
    name='Obbligazioni Tax',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Obbligazioni Tax_Percentage'],
    color='rgba(247, 238, 127, 0.3)',
)

add_bar_trace_percentage(
    fig,
    name='Azioni ed ETF Netto',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Azioni ed ETF (Netto)_Percentage'],
    color='rgba(219, 64, 82, 0.7)',
)

add_bar_trace_percentage(
    fig,
    name='Azioni ed ETF Tax',
    x=dataframe['Anno_Mese_str'],
    y=dataframe['Azioni ed ETF Tax_Percentage'],
    color='rgba(219, 64, 82, 0.3)',
)

# Update layout for 100% stacked bars
fig.update_layout(
    barmode='stack',
    title='Portfolio Composition Over Time (Percentage)',
    xaxis_title='Month',
    yaxis_title='Percentage (%)',
    legend_title='Categories',
    template='plotly_white',
    xaxis={
        'tickformat': '%Y-%m',
        'tickangle': -45,
        'type': 'category',
    },
    uniformtext={'minsize': 8, 'mode': 'hide'},
    margin={'b': 150, 't': 150},
    width=1980,
    height=1080,
)

# Set y-axis to range from 0 to 100%
fig.update_yaxes(range=[0, 100])

# Adjust the layout to ensure bars are ordered correctly
fig.update_xaxes(categoryorder='category ascending')

# Optionally, add a total label (100%) on top of each bar
fig.add_trace(
    go.Scatter(
        x=dataframe['Anno_Mese_str'],
        y=[100] * len(dataframe),
        mode='text',
        text=['100%'] * len(dataframe),
        textposition='top center',
        textfont={'size': 12, 'color': 'black', 'family': 'Arial'},
        showlegend=False,
    )
)

# Show the figure
fig.show()